In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import cm

def Resanalysis(optreg,trnX,trny,tstX,tsty):    
    otstpred=pd.Series(optreg.predict(tstX),dtype=float)
    otrnpred=pd.Series(optreg.predict(trnX),dtype=float)
    r2 = r2_score(tsty,otstpred) 
    pear = scipy.stats.pearsonr(tsty,otstpred)
    rmser = (mean_squared_error(tsty,otstpred))**0.5
    spmn = scipy.stats.spearmanr(tsty,otstpred)
    return [r2,pear[0],rmser,spmn[0]] # rval causes error, so replaced with dummy number 

In [ ]:
import scipy
from sklearn.model_selection import GridSearchCV

def findbestesti(esti,parm,trnX,trny):
    grisea=GridSearchCV(esti,parm,scoring='neg_root_mean_squared_error',cv=8)
    modl=grisea.fit(trnX,trny)
    print('training set R-pearson = ', scipy.stats.pearsonr(trny,modl.predict(trnX)))
    return modl.best_estimator_

In [ ]:
def elim_high_corr(X_df, corr_thresh,y):
  X_corr = X_df.corr()
  highcorr = []
  for i in X_df.columns:
    for j in X_df.columns:
      if i!=j and abs(X_corr[i][j])>corr_thresh:
        if (X_df[i].std() <= X_df[j].std()) and i not in highcorr:
          highcorr += [i]
        elif (X_df[i].std() > X_df[j].std()) and j not in highcorr:
          highcorr += [j]
  print("columns to drop: ", len(highcorr),"\n", highcorr)
  print("reduced no. of columns: ", X_df.shape[1]-len(highcorr))
  Xreduced = X_df.drop(highcorr,axis=1)
  return [Xreduced, highcorr]

In [ ]:
def fill_pred_dict(dic, spl, nf, ytest, ypred):
  dic['test_split-'+str(spl)+'_features-'+str(nf)] = np.array(y_test)
  dic['pred_split-'+str(spl)+'_features-'+str(nf)] = np.array(ypred)
  return dic

In [ ]:
def plotnsave(est, spl, nf, y_test, y_pred):
  plt.figure(figsize=(10,10))
  plt.scatter(y_test, y_pred)
  plt.plot(np.linspace(-9,-2), np.linspace(-9,-2), c='black')
  ttl = 'Split'+str(spl)+'_'+str(nf)+'Features_'+ est
  plt.title(ttl)
  plt.xlim(-9,-2)
  plt.ylim(-9,-2)
  plt.xlabel('y_test (kcal/mol)',size=20)
  plt.ylabel('y_pred (kcal/mol)',size=20)
  plt.savefig('Split'+str(spl)+'_Features'+str(nf)+'_'+est+'.png')

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.inspection import permutation_importance
from sklearn.dummy import DummyRegressor

In [ ]:
DF00 = pd.read_excel('DF_1174X50_04-2023.xlsx',index_col=0)

#remv = 16211131
#DF00 = DF00.drop(index = remv)
#print(DF00.shape)
DF01 = DF00[DF00['Molecular Weight']<1000]
#print(DF01.shape)
DF02 = DF01[DF01['Molecular Weight']>100]
#print(DF02.shape)
DF03 = DF02[DF02['Hawkins_GBSA_Score']<0]
print(DF03.shape)
DF03.columns[-12:]

In [ ]:
from sklearn.preprocessing import RobustScaler as RS

y = DF03['Hawkins_GBSA_sa_energy']

todrop = DF03.columns[-9:]
X0 = DF03.drop(todrop,axis=1)

vt = VarianceThreshold(0.01)
X1 = pd.DataFrame(vt.fit_transform(X0), index=X0.index, columns=X0.columns[vt.get_support()])
print(X1.shape)

X2 = elim_high_corr(X1, 0.7, y)[0]

In [ ]:
numoffea = [2,4,6,8,10,12,X2.shape[1]]   #[10, X_std.shape[1]]     #[X_std.shape[1]]

r2dfnam = []
pdfnam = []
rmdfnam = []
rmsedfnam = []
spmndfnam = []
for nf in numoffea:
    r = 'RF'+str(nf)
    g = 'GBR'+str(nf)
    s = 'SVR'+str(nf)
    l = 'LR'+str(nf)
    r2dfnam += [r, g, s, l]
    pdfnam += [r, g, s, l] 
    rmsedfnam += [r, g, s, l]
    spmndfnam += [r, g, s, l]
print(pdfnam)

spl_is = []
spl_is += ['split_'+str(i) for i in range(10)]
feaimpdf = pd.DataFrame(0, index=spl_is, columns=X2.columns)
r2df = pd.DataFrame(index=spl_is, columns=r2dfnam)
peardf = pd.DataFrame(index=spl_is, columns=pdfnam)
rmsedf = pd.DataFrame(index=spl_is, columns=rmsedfnam)
spmndf = pd.DataFrame(index=spl_is, columns=spmndfnam)

In [ ]:
lin_dict_pred = {}
gbr_dict_pred = {}
svr_dict_pred = {}
rfr_dict_pred = {}

In [ ]:
ntfs = [100,150,200,250]
maxfeafs =['sqrt','log2','auto']
minsmplffs = [1,3,5]
parfs = {'max_features':maxfeafs, 'n_estimators':ntfs, 'min_samples_leaf':minsmplffs}

ne = [100,200,300,400,500]
minsmpleaf = [1,3,5]
mxf =  ['sqrt','log2','auto']
rfpar = {'n_estimators':ne, 'max_features':mxf, 'min_samples_leaf':minsmpleaf} 

gbrne = [100,150,200,250]
gbrpar = {'max_features':mxf, 'n_estimators':gbrne, 'min_samples_leaf':minsmpleaf}

c = np.logspace(-1,1,15)
gam = ['scale','auto']
ker = ['rbf']  
svrpar = {'C':c, 'gamma':gam, 'kernel':ker}

In [ ]:
##### Test-Train Split #####
from sklearn.model_selection import train_test_split
rnd = 17061991

for spl in range(10):
    print('\n')
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2,random_state=rnd+(43210*spl))
    print('Currently running Split ',spl)
    
    gbrfs = GradientBoostingRegressor(random_state=rnd+(54321*spl))
    gscv = GridSearchCV(gbrfs, parfs, scoring='neg_root_mean_squared_error', cv=10)
    gscvfit = gscv.fit(X_train,y_train)
    bstest = gscvfit.best_estimator_
    feaimp = bstest.feature_importances_

    print("Best estimator for feature selection: ", bstest)
    print("Best estim. Training Preasons R: ", scipy.stats.pearsonr(y_train,bstest.predict(X_train)))

    feaimpdf.loc['split_'+str(spl)] = feaimp 
    sortfi = np.sort(feaimp)

    for nf in numoffea:
        print("\n\nNumber Of Features :", nf)
        topfea =  X2.columns[feaimp>=sortfi[-nf]]
        
        X_trn = X_train[topfea]
        X_tst = X_test[topfea]
        
        linr = LinearRegression()
        lrfit = linr.fit(X_trn,y_train)
        lrr = Resanalysis(lrfit,X_trn,y_train,X_tst,y_test)
        r2df.iloc[spl]['LR'+str(nf)] = lrr[0]
        peardf.iloc[spl]['LR'+str(nf)] = lrr[1]
        rmsedf.iloc[spl]['LR'+str(nf)] = lrr[2]
        spmndf.iloc[spl]['LR'+str(nf)] = lrr[3]
        lin_dict_pred = fill_pred_dict(lin_dict_pred, spl, nf, y_test, lrfit.predict(X_tst))
        plotnsave('LR',spl,nf,y_test, lrfit.predict(X_tst))

        svr = SVR()
        optsvr = findbestesti(svr,svrpar,X_trn,y_train)
        svrr = Resanalysis(optsvr,X_trn,y_train,X_tst,y_test)
        r2df.iloc[spl]['SVR'+str(nf)] = svrr[0]
        peardf.iloc[spl]['SVR'+str(nf)] = svrr[1]
        rmsedf.iloc[spl]['SVR'+str(nf)] = svrr[2]
        spmndf.iloc[spl]['SVR'+str(nf)] = svrr[3]
        svr_dict_pred = fill_pred_dict(svr_dict_pred, spl, nf, y_test, optsvr.predict(X_tst))
        plotnsave('SVR',spl,nf,y_test, optsvr.predict(X_tst))
        
        gbr = GradientBoostingRegressor(random_state=rnd+(65432*spl))
        optgbr = findbestesti(gbr,gbrpar,X_trn,y_train)
        print(optgbr)
        gbrfr = Resanalysis(optgbr,X_trn,y_train,X_tst,y_test)
        r2df.iloc[spl]['GBR'+str(nf)] = gbrfr[0]
        peardf.iloc[spl]['GBR'+str(nf)] = gbrfr[1]
        rmsedf.iloc[spl]['GBR'+str(nf)] = gbrfr[2]
        spmndf.iloc[spl]['GBR'+str(nf)] = gbrfr[3]
        gbr_dict_pred = fill_pred_dict(gbr_dict_pred, spl, nf, y_test, optgbr.predict(X_tst))
        plotnsave('GBR',spl,nf,y_test, optgbr.predict(X_tst))

        rfr = RandomForestRegressor(random_state= rnd+(76543*spl))
        optrfr = findbestesti(rfr,rfpar,X_trn,y_train)
        print(optrfr)
        rfrr = Resanalysis(optrfr,X_trn,y_train,X_tst,y_test)
        r2df.iloc[spl]['RF'+str(nf)] = rfrr[0]
        peardf.iloc[spl]['RF'+str(nf)] = rfrr[1]
        rmsedf.iloc[spl]['RF'+str(nf)] = rfrr[2]
        spmndf.iloc[spl]['RF'+str(nf)] = rfrr[3]
        rfr_dict_pred = fill_pred_dict(rfr_dict_pred, spl, nf, y_test, optrfr.predict(X_tst))
        plotnsave('RFR',spl,nf,y_test, optrfr.predict(X_tst))

    print(peardf)
    del optgbr, optrfr, optsvr   
    del X_train, X_test, y_train, y_test

In [ ]:
peardf.to_excel('RPearson_DF_fin_.xlsx')
r2df.to_excel('R2_DF_fin_.xlsx')
rmsedf.to_excel('RMSE_DF_fin_.xlsx')
spmndf.to_excel('SpearmanCof_DF_fin_.xlsx')
feaimpdf.to_excel('FeatureImpr_DF_fin_.xlsx')

In [ ]:
(pd.DataFrame.from_dict(lin_dict_pred)).to_excel('LinReg_ypred-vs-ytest_.xlsx')
(pd.DataFrame.from_dict(gbr_dict_pred)).to_excel('GBReg_ypred-vs-ytest_.xlsx')
(pd.DataFrame.from_dict(svr_dict_pred)).to_excel('SVReg_ypred-vs-ytest_.xlsx')
(pd.DataFrame.from_dict(rfr_dict_pred)).to_excel('RFReg_ypred-vs-ytest_.xlsx')